# 판다스 코드정리

## 위도와 경도에 111을 곱해 반경에 있는 발전소를 찾는 코드

In [ ]:
for i in new_daily.plant_id[new_daily.plant_id.isin(plant_list.plant_id)].unique():
    for t in range(1,31):
        target_lat = new_daily.latitude[new_daily.plant_id==i].values[0]
        target_long = new_daily.longitude[new_daily.plant_id==i].values[0]
        target_pv = new_daily.avg_pv_time[new_daily.plant_id==i].values[0]
        df_pm2 = new_daily[new_daily.plant_id!=i]
        df_pm2['target_plant_id'] = i
        df_pm2['distance']=np.sqrt((df_pm2.latitude-target_lat)**2+(df_pm2.longitude-target_long)**2)*111
        df_pm3 = df_pm2[df_pm2.distance<=t].sort_values(['distance'])
        legion_avg = np.nanmean(df_pm3.avg_pv_time[(df_pm3.avg_pv_time>0)&(df_pm3.avg_pv_time<10)])
        if len(df_pm3)>30:
            print(i,"  :   ", np.round(target_pv/legion_avg,2),"발전소 수: ",len(df_pm3), "반경:  " ,t)
            plant_id +=[i]
            pvtime_percent +=[np.round(target_pv/legion_avg,2)]
            plant_count +=[len(df_pm3)]
            plant_range +=[t]
            break
        if t==30 and len(df_pm3)<30:
            no+=[i]
            plant_id +=[i]
            pvtime_percent +=[np.round(target_pv/legion_avg,2)]
            plant_count +=[len(df_pm3)]
            plant_range +=[t]

## 지역별 통계를 위해 통계값 기반 전처리 코드

In [ ]:
gld_threshold_2020_low= gld_2020[['base_date','location_group_code','avg_pv_time']].groupby(['base_date','location_group_code']).apply(lambda x: x.median()-3*1.4826*median_absolute_deviation(x)).reset_index()

gld_threshold_2020_up= gld_2020[['base_date','location_group_code','avg_pv_time']].groupby(['base_date','location_group_code']).apply(lambda x: x.median()+3*1.4826*median_absolute_deviation(x)).reset_index()

gld_threshold_2020_up.columns =['base_date', 'location_group_code', 'threshold_upper']
gld_threshold_2020_low.columns =['base_date', 'location_group_code', 'threshold_lower']

gld_2020_2 = pd.merge(gld_2020,gld_threshold_2020_low,on=['location_group_code','base_date'])

gld_2020_2 = pd.merge(gld_2020_2,gld_threshold_2020_up,on=['location_group_code','base_date'])

gld_2020_3 = gld_2020_2[(gld_2020_2.avg_pv_time>=gld_2020_2.threshold_lower)&(gld_2020_2.avg_pv_time<gld_2020_2.threshold_upper)].reset_index(drop=True)

day_2020= pd.DataFrame(Counter(gld_2020_3.plant_id).items(),columns=['plant_id','day_count'])

final_list_2020= list(day_2020.plant_id[day_2020.day_count>=365])

gld_2020_4 = pd.merge(gld_2020_3,day_2020,on='plant_id')

gld_2020_5 = gld_2020_4[gld_2020_4.day_count>=365].reset_index(drop=True)

gld_2020_5['month']=[x[:7] for x in gld_2020_5.base_date]

gld_2020_5.location_group_code=[np.int(x) for x in gld_2020_5.location_group_code]

gld_2020_6= gld_2020_5.pivot_table(index=['plant_id','month'],values=['avg_pv_time','location_group_code','pv_capacity','day_count'],aggfunc=np.mean).reset_index()

gld_2020_6.location_group_code = [str(x) for x in gld_2020_6.location_group_code]



gld_2020_6_pv_amount = gld_2020_5.pivot_table(index=['plant_id','month'],values='pv_amount',aggfunc=np.sum).reset_index()

gld_2020_7 = pd.merge(gld_2020_6,gld_2020_6_pv_amount,on=["plant_id","month"])

gld_2020_8 = pd.merge(gld_2020_7,sido,on='location_group_code')



len(gld_2020_8.plant_id.unique())

sum(gld_2020_8.drop_duplicates(['plant_id']).pv_capacity)

gld_2020_9= pd.merge(gld_2020_8,pm[['plant_id','address']],on='plant_id')

gld_2020_9.columns = ['발전소id','월','발전시간','발전일수','지역코드','발전용량','총발전량','지역구분','주소']

In [ ]:
def getGPS(addr):
    addr = str(addr)
    if ((addr.split()[-1]=='.') | (addr.split()[-1]=='-')):
        addr = ' '.join(addr.split()[:-1])
    count =0
    is_bad = False
    with requests.Session() as s:
        while count ==0:
            params = {'X-NCP-APIGW-API-KEY-ID' : ID, 'X-NCP-APIGW-API-KEY' : KEY,
                      'query' : addr}
            response = s.get('https://naveropenapi.apigw.ntruss.com/map-geocode/v2/geocode', params=params)
            result = response.json()
            if result['status']!='OK':
                print('have no adress')
                return {130, 37, True}
            count = result['meta']['totalCount']
            if count !=0:
                lon = result['addresses'][0]['x']
                lat = result['addresses'][0]['y']
                roadadrr = result['addresses'][0]['roadAddress']
                jibunaddr = result['addresses'][0]['jibunAddress']
                #                 print(lon)
                ret = {'count' : count, 'lon' : lon, 'lat' : lat, 'road' : roadadrr, 'jibun' : jibunaddr, 'is_bad':is_bad}
                #                 print(lon, lat, is_bad)
                return [np.float32(lon), np.float32(lat), is_bad]
            addr= ' '.join(addr.split()[:-1])

## 다양한 전처리 내용. feature를 후행, 선행으로 만드는 작업도 포함되어 있음

In [ ]:
df2 = df.sort_values(['analysis_time']).reset_index(drop=True)
df2['analysis_time_kst'] = [x.astimezone(pytz.timezone('Asia/Seoul')) for x in df2.analysis_time]

df2['lead_hour_timedelta'] = [timedelta(hours=x) for x in df2.lead_hour]
df2['base_time'] = df2.analysis_time_kst + df2.lead_hour_timedelta
df2['base_hour'] = [x.hour for x in df2.base_time]
df2['base_date'] = [x.date().strftime('%Y-%m-%d') for x in df2.base_time]
df2['analysis_hour'] = [x.time().hour for x in df2.analysis_time]
df_0 = df2[(df2.analysis_hour.isin([0])) & (df2.lead_hour >= 15)].drop_duplicates(['base_date', 'base_hour'],
                                                                                  keep='last').sort_values(
    ['base_date', 'base_hour']).reset_index(drop=True)
df_0_result = df_0[(df_0.base_date >= '2021-10-01') & (df_0.base_date < '2021-11-01')].reset_index(drop=True)
df_0_result = df_0_result[
    ['base_date', 'base_hour', 'total_shortwave', 'temperature', "humidity_relative", "pressure_msl", "cloud_total_max",
     "analysis_time_kst", 'lead_hour']]
df_0_result['total_shortwave_lead_1'] = df_0_result.total_shortwave.shift(
    -1).fillna(0)
df_0_result['sunny_radiation'] = sunny_rad.IC

df_0_result.temperature -= 273.15
df_0_result.pressure_msl /= 100
df_0_result.cloud_total_max *= 10
df_0_result['sunny_radiation_lead_1'] = df_0_result.sunny_radiation.shift(
    -1).fillna(0)
df_0_result['temperature_lead_1'] = df_0_result.temperature.shift(-1).fillna(
    method="ffill")
df_0_result['humidity_relative_lead_1'] = df_0_result.humidity_relative.shift(
    -1).fillna(
    method="ffill")
df_0_result['pressure_msl_lead_1'] = df_0_result.pressure_msl.shift(
    -1).fillna(
    method="ffill")
df_0_result['cloud_total_max_lead_1'] = df_0_result.cloud_total_max.shift(
    -1).fillna(
    method="ffill")
df_0_result['total_shortwave_lead_2'] = df_0_result.total_shortwave.shift(
    -2).fillna(0)

## seq 데이터셋 만드는 것

In [ ]:
train_x = []
train_y = []
base_date = []
base_hour = []
for i in range(160,len(train)-160):
    train_x+=[np.array(np.array(train[feature][i:i+160]).transpose())]
    train_y+=[train.pptot.values[i:i+160]]
    base_date +=[train.base_date.values[i:i+160]]
    base_hour+=[train.base_time.values[i:i+160]]

## lag 변수 만들기

In [ ]:
new_df = pd.DataFrame()
for i in df.category.unique():
    df_ = df[df.category==i]
    df_['pptot_lag_160'] = df_.pptot.shift(160)
    df_['pptot_lag_96'] = df_.pptot.shift(96)
    df_['pptot_lag_48'] = df_.pptot.shift(48)
    df_['pptot_lag_24'] = df_.pptot.shift(24)
    df_['pptot_lag_12'] = df_.pptot.shift(12)
    df_['pptot_lag_6'] = df_.pptot.shift(6)
    df_ = df_[160:]
    new_df = new_df.append(df_)